# Imports

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'

In [2]:
import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import tensorflow as tf
if tf.test.is_gpu_available() & tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    #from tensorflow.python.client import device_lib
    #print(device_lib.list_local_devices())

#Original: from keras import callbacks, backend as K
from tensorflow.compat.v1.keras import callbacks, backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.utils import multi_gpu_model
from keras.initializers import glorot_uniform
from keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad
from keras.layers.advanced_activations import ReLU, LeakyReLU

from datetime import datetime
import pandas as pd
import numpy as np
import shutil

from numpy.random import seed
seed(1)
#Original: from tensorflow import set_random_seed
#Original: set_random_seed(1)
tf.random.set_seed(1)

#Original config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
#Original sess = tf.Session(config=config)
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

Using TensorFlow backend.


The installed version of TensorFlow 2.0.0 includes GPU support.



# Hilfsfunktionen

### Benutzerdefinierte Kostenfunktion & Metrik

In [3]:
def circular_mse(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.square(K.minimum(K.abs(y_pred - y_true), max_error - K.abs(y_pred - y_true))), axis=-1)

def circular_mae(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.minimum(K.abs(y_pred - y_true), K.abs(max_error - K.abs(y_pred - y_true))), axis=-1)

### Generierung Datenpipeline (Angepasst für Talos)

In [4]:
def create_data(batch_size, num_samples):

    df = pd.read_csv(_CSV_FILE)
    df_shuffled = df.sample(frac=1, random_state=1)
    df_train = df_shuffled[0:int(num_samples*0.8//batch_size*batch_size)]   
    df_valid = df_shuffled.drop(df_shuffled.index[0:df_train.shape[0]])[0:int(num_samples*0.2//batch_size*batch_size)]
    
    if _USE_REAL_TRAIN_DATA:
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac=1, random_state=1)
        df_shuffled_real = df_shuffled_real.drop(df_shuffled_real.index[(df_shuffled_real.shape[0] - 61):df_shuffled_real.shape[0]])
        df_train_real = df_shuffled_real[0:int(df_shuffled_real.shape[0]*0.8//batch_size*batch_size)]   
        df_valid_real = df_shuffled_real.drop(df_shuffled_real.index[0:df_train_real.shape[0]])
        df_train = df_train.drop(df_train.index[df_train.shape[0] - df_train_real.shape[0]:df_train.shape[0]])
        df_valid = df_valid.drop(df_valid.index[df_valid.shape[0] - df_valid_real.shape[0]:df_valid.shape[0]])
        df_train = df_train.append(df_train_real)
        df_valid= df_valid.append(df_valid_real)
        
    if _USE_DATA_AUGMENTATION:
        train_data_generator = ImageDataGenerator(
            rescale=1./255
            ,width_shift_range=0.1
            ,height_shift_range=0.1
            ,zoom_range=0.1
            ,brightness_range=(0.25, 0.75)
            ,fill_mode='nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale=1./255
        )
    
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe=df_train
        ,directory=_IMAGE_DIR
        ,x_col='Filename'
        ,y_col=['Elevation', 'Azimuth']
        ,class_mode='other'
        ,target_size=(224, 224)
        ,color_mode='rgb'
        ,shuffle=True
        ,seed=77
        ,batch_size=batch_size
    )
    
    valid_data_generator = ImageDataGenerator(rescale=1./255)
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe=df_valid
        ,directory=_IMAGE_DIR
        ,x_col='Filename'
        ,y_col=['Elevation', 'Azimuth']
        ,class_mode='other'
        ,target_size=(224, 224)
        ,color_mode='rgb'
        ,shuffle=False
        ,seed=77
        ,batch_size=batch_size
    )
    
    return train_generator, valid_generator

### Generierung Modell (Angepasst für Talos)

In [5]:
def grid_model_fine(x, y, x_val, y_val, params):
    
    K.clear_session()
    
    train_generator, valid_generator = create_data(params['batch_size'], params['samples'])
    
    dropout_rate = params['dropout']
    first_neuron = params['first_neuron']
    
    if params['activation'] == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = params['leaky_alpha'])
    elif params['activation'] == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    for layer in cnn.layers[:15]:
        layer.trainable = False
        #print(layer.name, layer.trainable)
    
    model.add(cnn)
    
    fc = Sequential()
    fc.add(Flatten(input_shape=model.output_shape[1:])) # (7, 7, 512)
    
    fc.add(Dense(units=first_neuron, kernel_initializer=glorot_uniform(seed=1)))
    fc.add(activation_layer)
    if dropout_rate > 0.0:
        fc.add(Dropout(rate=dropout_rate))
    
    hidden_neuron_fraction = first_neuron
    for i in range(params['hidden_layers']):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        fc.add(Dense(units=hidden_neuron_fraction, kernel_initializer=glorot_uniform(seed=1)))
        fc.add(activation_layer)
        if dropout_rate > 0.0:
            fc.add(Dropout(rate=dropout_rate))
    
    fc.add(Dense(units=2, kernel_initializer=glorot_uniform(seed=1)))
    fc.load_weights(_MODEL_DIR + _MODEL_TO_LOAD)
    model.add(fc)
    
    model.compile(
        optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])*1e-2)
        ,loss=circular_mse if(_LOSS == 'CMSE') else 'mean_squared_error'
        ,metrics=[circular_mae]
    )
    
    checkpointer = callbacks.ModelCheckpoint(
        filepath=_LOG_DIR + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(_MODEL_TO_LOAD_INDEX, train_generator.n)
        ,monitor='val_circular_mae'
        #,monitor='val_loss'
        ,verbose=1
        ,save_weights_only=False
        ,save_best_only=True
        ,mode='min'
    )
    
    csv_logger = callbacks.CSVLogger(
        filename=_LOG_DIR + 'CNN_Base_{}_Logger_{}.csv'.format(_MODEL_TO_LOAD_INDEX, train_generator.n)
        ,separator=','
        ,append=False
    )

    lr_reducer = callbacks.ReduceLROnPlateau(
        #monitor='val_circular_mae'
        monitor='val_loss'
        ,factor=0.1
        ,patience=10
        ,verbose=1
        ,mode='min'
        ,min_delta=0.0001
    )
    
    early_stopper = callbacks.EarlyStopping(
        #monitor='val_circular_mae'
        monitor='val_loss'
        ,min_delta=0
        ,patience=15
        ,verbose=1
        ,mode='min'
    )
     
    out = model.fit_generator(
        generator=train_generator
        ,steps_per_epoch=train_generator.n//train_generator.batch_size
        ,validation_data=valid_generator
        ,validation_steps=valid_generator.n//valid_generator.batch_size
        ,callbacks=[checkpointer
                    ,csv_logger
                    ,lr_reducer
                    ,early_stopper
                   ]
        ,epochs=params['epochs']
        ,workers=8
    )
    
    return out, model

# Parameter

### Dateisystem

In [10]:
_RUN = '03'
_LOSS = 'CMSE'
_DATASET_NAME = 'Dataset_2019-08-13'

In [11]:
_USE_REAL_TRAIN_DATA = False
_USE_DATA_AUGMENTATION = False

In [12]:
#Original: _IMAGE_DIR = 'D:\\tkroiss\\Datasets\\{}\\'.format(_DATASET_NAME)
_IMAGE_DIR = '..\\data-target\\datasets\\{}\\'.format(_DATASET_NAME)
_CSV_FILE = _IMAGE_DIR + 'images.csv'
_CSV_FILE_REAL = _IMAGE_DIR + 'images_real.csv'

_DEVICE = 'RTX' if(os.environ["CUDA_VISIBLE_DEVICES"]=='0') else 'TITAN'

#Original: _MODEL_DIR = 'E:\\tkroiss\\{}_Regression_{}\\Base\\'.format(_RUN, _LOSS)
#_LOG_DIR = 'E:\\tkroiss\\{}_Regression_{}\\Top_1_DA_Synth\\Train\\'.format(_RUN, _LOSS)
_MODEL_DIR = '..\\models-target\\{}_Regression_{}\\Base\\'.format(_RUN, _LOSS)
_LOG_DIR = '..\\logs\\{}_Regression_{}\\Top_1_DA_Synth\\Train\\'.format(_RUN, _LOSS)
assert(os.path.exists(_LOG_DIR)) == False, 'Vorsicht, Verzeichnis existiert bereits!'
os.makedirs(_LOG_DIR)

### Top 3 FC-Gewichte

In [13]:
df = pd.read_csv(_MODEL_DIR + 'Talos_Results_Base.csv').drop(columns=['round_epochs', 'samples', 'epochs', 'dropout'], axis=0)
df = df.sort_values('val_circular_mae', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
df[:3]

,val_loss,val_circular_mae,loss,circular_mae,activation,batch_size,first_neuron,hidden_layers,lr,optimizer
1,606.268921,12.967867,1289.508174,22.522382,leakyrelu,64,1024,3,1,<class 'keras.optimizers.Adam'>
0,703.312073,13.492824,1239.709859,21.308022,leakyrelu,64,1024,2,1,<class 'keras.optimizers.Adam'>
2,695.030945,13.861068,1254.469433,21.318181,leakyrelu,64,2048,2,1,<class 'keras.optimizers.Adam'>


### GridSerach

In [14]:
def get_params(top_results_index):
    
    p = {'samples':[100000],
         'epochs':[200],
         'batch_size':[df.iloc[top_results_index]['batch_size']],
         'optimizer':[Adam],
         'lr':[df.iloc[top_results_index]['lr']],
         'first_neuron':[df.iloc[top_results_index]['first_neuron']],
         'dropout':[0.25],
         'activation':[df.iloc[top_results_index]['activation']],
         'leaky_alpha':[0.1], #Default bei LeakyReLU, sonst PReLU
         'hidden_layers':[df.iloc[top_results_index]['hidden_layers']]
        }
    
    return p

In [15]:
def get_params(top_results_index):
    
    p = {'samples':[20000, 40000, 100000],
         'epochs':[200],
         'batch_size':[df.iloc[top_results_index]['batch_size']],
         'optimizer':[Adam],
         'lr':[df.iloc[top_results_index]['lr']],
         'first_neuron':[df.iloc[top_results_index]['first_neuron']],
         'dropout':[0.25],
         'activation':[df.iloc[top_results_index]['activation']],
         'leaky_alpha':[0.1], #Default bei LeakyReLU, sonst PReLU
         'hidden_layers':[df.iloc[top_results_index]['hidden_layers']]
        }
    
    return p

### Start Talos

In [16]:
dummy_x = np.empty((1, 2, 3, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
#     for top_results_index in range(3):
#     for top_results_index in [1]:
        top_results_index = 0
        _MODEL_TO_LOAD_INDEX = df.iloc[top_results_index].name
        _MODEL_TO_LOAD = 'Best_Weights_FC_{}.hdf5'.format(_MODEL_TO_LOAD_INDEX)

        _TMP_DIR = 'TMP_TALOS_{}'.format(_DEVICE)
        _CSV_RESULTS = _LOG_DIR + 'Talos_Results_Fine.csv'

        startTime = datetime.now()

        t = ta.Scan(
            x=dummy_x
            ,y=dummy_y
            ,model=grid_model_fine
            ,params=get_params(top_results_index)
            ,experiment_name= _TMP_DIR
            #,shuffle=False
            ,reduction_metric = 'val_circular_mae'
            ,disable_progress_bar=False
            ,print_params = True
            ,clear_session=True
        )

        print("Time taken:", datetime.now() - startTime)

        df_experiment_results = pd.read_csv(_TMP_DIR + '\\' + os.listdir(_TMP_DIR)[0])
        df_experiment_results['Base'] = None
        for i in range(df_experiment_results.shape[0]):
            df_experiment_results['Base'][i] = _MODEL_TO_LOAD_INDEX

        if os.path.isfile(_CSV_RESULTS):
            df_experiment_results.to_csv(_CSV_RESULTS, mode='a', index=False, header=False)
        else:
            df_experiment_results.to_csv(_CSV_RESULTS, index=False)

        shutil.rmtree(_TMP_DIR)

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

{'activation': 'leakyrelu', 'batch_size': 64, 'dropout': 0.25, 'epochs': 200, 'first_neuron': 1024, 'hidden_layers': 3, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'keras.optimizers.Adam'>, 'samples': 20000}


ValueError: Invalid class_mode: other; expected one of: {'binary', 'multi_output', 'input', 'sparse', None, 'categorical', 'raw'}